In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

In [ ]:
number_of_inputs = 28 * 28
n_hidden1 = 100
n_hidden2 = 100
n_output = 10

def neuron_layer(X, n_neurons, name):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        # truncated normal distributions limit the size of the weights, speeding up the training time.
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        return tf.nn.relu(z)
        
x = tf.placeholder(tf.float32, shape=(None, number_of_inputs), name="input")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1_output = neuron_layer(x, n_hidden1, "hidden1")
    hidden2_output = neuron_layer(hidden1_output, n_hidden2, "hidden2")
    logits = neuron_layer(hidden2_output, n_output, "output")

with tf.name_scope("loss"):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(cross_entropy, name="loss")

with tf.name_scope("training"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    k = 1
    correctness = tf.nn.in_top_k(logits, y, k)
    accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32)) * 100

In [ ]:
epochs = 1
batch_size = 50
n_batches = int(np.ceil(mnist.train.num_examples // batch_size))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

interim_checkpoint_path = "./checkpoints/mnist_model.ckpt"

from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
log_dir = "{}/run-{}/".format(root_logdir, now)

loss_summary = tf.summary.scalar('loss', loss)
accuracy_summary = tf.summary.scalar("accuracy", accuracy)
summary_op = tf.summary.merge([loss_summary, accuracy_summary])
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    #saver.restore(sess, interim_checkpoint_path)
    
    for epoch in range(epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            if batch_index % 10 == 0:
                summary_str = summary_op.eval(feed_dict={x: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            t, l, a = sess.run([training_op, loss, accuracy], feed_dict={x: X_batch, y: y_batch})
            
            if batch_index == 0: print("loss:", l, "accuracy:", a)
        if epoch % 100 == 0:
            save_path = saver.save(sess, interim_checkpoint_path)

    save_path = saver.save(sess, "./checkpoints/mnist_model_final.ckpt")